# Scraper commands for publications on springer
## Examples

Here it is only about the steps, to parse are necessary so that it can be included in the end.

There are different website types for publications on springer.

In [209]:
journal_doi = 'https://doi.org/10.1007/s12525-020-00445-0'
journal_full_link = 'https://link.springer.com/article/10.1007/s12525-020-00445-0'

conference_chapter_link = 'https://link.springer.com/chapter/10.1007/978-3-030-49570-1_14'
conference_book_link = 'https://link.springer.com/book/10.1007/978-3-642-22531-4'

volume_contribution_link = 'https://link.springer.com/chapter/10.1007/978-3-030-06234-7_27'
#volume_link = 'https://link.springer.com/book/10.1007/978-3-030-06234-7'

springer_links = [journal_full_link, conference_chapter_link, conference_book_link, volume_contribution_link]
springer_links

['https://link.springer.com/article/10.1007/s12525-020-00445-0',
 'https://link.springer.com/chapter/10.1007/978-3-030-49570-1_14',
 'https://link.springer.com/book/10.1007/978-3-642-22531-4',
 'https://link.springer.com/chapter/10.1007/978-3-030-06234-7_27']

In [210]:
import requests
from bs4 import BeautifulSoup
import json

In [211]:
# identify the type of the link
def get_springer_link_type(url):
    if '/chapter/' in url:
        return 'chapter'
    elif '/book/' in url:
        return 'book'
    elif '/article/' in url:
        return 'article'
    else:
        return 'unknown'


In [212]:
for link in springer_links:
    print(get_springer_link_type(link))

article
chapter
book
chapter


In [213]:
def get_bs(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15'}
        r = requests.get(url, headers=headers)
        print(r.status_code)
        bs = BeautifulSoup(r.text, 'html.parser')
    except:
        print('Error: ', url)
        return None
    return bs

In [214]:
journal_soup = get_bs(journal_full_link)
conference_chapter_soup = get_bs(conference_chapter_link)
conference_book_soup = get_bs(conference_book_link)
volume_contribution_soup = get_bs(volume_contribution_link)
springer_soups = [journal_soup, conference_chapter_soup, conference_book_soup, volume_contribution_soup]

200
200
200
200


In [215]:
# get whole data from json and loads to dict
# articles need to be prefiltered because they are nested differently.'
def get_json_data(bs):
    json_string = bs.find('script', {'type': 'application/ld+json'}).text
    json_data = json.loads(json_string)

    if '{"mainEntity":' in json_string:
        return json_data['mainEntity']
    else:
        return json_data


Creating json files for the three different types of publications for testing

In [216]:
journal_json = get_json_data(journal_soup)
conference_chapter_json = get_json_data(conference_chapter_soup)
conference_book_json = get_json_data(conference_book_soup)
volume_contribution_json = get_json_data(volume_contribution_soup)
json_data_list = [journal_json, conference_chapter_json, conference_book_json, volume_contribution_json]

In [217]:
# get names of json fields
def get_json_fields(json_data):
    return list(json_data.keys())

In [218]:
get_json_fields(journal_json)

['headline',
 'description',
 'datePublished',
 'dateModified',
 'pageStart',
 'pageEnd',
 'license',
 'sameAs',
 'keywords',
 'image',
 'isPartOf',
 'publisher',
 'author',
 'isAccessibleForFree',
 '@type']

## Main Fields
Title #TODO get title also for book (if instruction or with json)

In [219]:
def get_title(bs):
    try:
        title = bs.find('h1', {'class': 'c-article-title'}).text
        return title
    except:
        return None

In [220]:
for soup in springer_soups:
    print(get_title(soup))

Exploring customers’ likeliness to use e-service touchpoints in brick and mortar retail
A Two-Phase Framework for Detecting Manipulation Campaigns in Social Media
None
Applications of Artificial Intelligence in Supply Chain Management and Logistics: Focusing Onto Recognition for Supply Chain Execution


### Authors

In [221]:
def get_authors(bs):
    """
    Return list of authors in the format:
    [{'name': 'Author Name', 'orcid': orcid}, ...]

    :param bs: Received bs of the publication
    :return: list of dicts
    """
    try:
        json_data = get_json_data(bs)
        authors = []
        for author in json_data.get('author'):
            name = author.get('name')
            # split name at comma and reverse
            name = name.split(', ')
            name = name[1] + ' ' + name[0]
            orcid = author.get('url')
            authors.append({'name': name,
                            'orcid': orcid})
        return authors
    except:  # if no author is found you cannot iterate over None
        return None

In [222]:
for soup in springer_soups:
    print(get_authors(soup))

[{'name': 'Benjamin Barann', 'orcid': 'http://orcid.org/0000-0002-1965-2688'}, {'name': 'Jan H. Betzing', 'orcid': None}, {'name': 'Marco Niemann', 'orcid': None}, {'name': 'Benedikt Hoffmeister', 'orcid': None}, {'name': 'Jörg Becker', 'orcid': None}]
[{'name': 'Dennis Assenmacher', 'orcid': None}, {'name': 'Lena Clever', 'orcid': None}, {'name': 'Janina Susanne Pohl', 'orcid': None}, {'name': 'Heike Trautmann', 'orcid': None}, {'name': 'Christian Grimme', 'orcid': None}]
None
[{'name': 'Bernd Hellingrath', 'orcid': None}, {'name': 'Sandra Lechtenberg', 'orcid': None}]


### Keywords #TODO may be replaced with BS scraping because weird data structure

In [223]:
def get_keywords(bs):
    """
    Return list of keywords from json data
    :param bs: Received bs of the publication
    :return:
    """

    """
    Return list of keywords in the format:
    [keyword1, keyword2, ...]
    :param bs: Received bs of the publication
    :return: list: String
    """
    try:
        json_data = get_json_data(bs)
        keywords_string = json_data.get('keywords')
        keywords = keywords_string.split(',')
        return keywords
    except:
        print("Error: no keywords found")
        return None

In [224]:
for soup in springer_soups:
    print(get_keywords(soup))

['IT in Business', 'e-Commerce/e-business']
['Social campaign detection', ' Stream clustering', ' Unsupervised learning']
['Java', ' XQuery', ' abstract interpretation', ' higher-order patterns', ' non-deterministic functions']
['']


### Abstract

In [225]:
def get_abstract(bs, url):
    """
    Returns the abstract of the articles, books/proceedings do not have abstracts.
    :param bs: Received bs of the publication
    :param url: Received url of the publication
    :return: String
    """
    if '/book/' in url:
        return None

    try:
        json_data = get_json_data(bs)
        abstract = json_data.get('description')
        return abstract
    except:
        print("Error: no abstract found")
        return None


In [226]:
for soup, url in zip(springer_soups, springer_links):
    print(get_abstract(soup, url))
    print("-------")

E-commerce has embraced the digital transformation and innovated with e-service touchpoints to improve customers’ experiences. Now some traditional, less-digitalized brick and mortar (BaM) retailers are starting to counteract the increasing competition by adopting digital touchpoints. However, the academic literature offers little in terms of what determines customers’ behavioral intentions toward e-service touchpoints. Therefore, drawing from the dominant design theory, this article first conceptually adapts selected dominant touchpoints of leading e-commerce solutions to BaM retail. Then 250 shoppers are surveyed regarding the likeliness that they will use the selected touchpoints, followed by an exploratory factor analysis to determine the touchpoints’ characteristics that lead to the shoppers’ assessments. The results suggest that customers prefer touchpoints that support product search and selection, provide information, and increase shopping efficiency. The likeliness that survey

### Pdf #todo #TODO: download

In [227]:
def get_pdf(bs, url):
    """
    Returns the pdf link of the publication, if available. Download might require login. Sometimes the pdf is not in the soup object unfortunately
    :param bs: Received bs of the publication
    :param url:
    :return:
    """
    try:
        pdf = None
        #todo automate download
        # differentiate between article, chapter and book
        if '/article/' in url:
            pdf = bs.find('div', class_='c-pdf-container').find('a', {'data-article-pdf': 'true'}).get('href')

        elif '/chapter/' in url:
            pdf_box = bs.find('div', {'class': 'c-article-access-provider'})
            pdf = pdf_box.find('a', {'data-track-action': 'Pdf download'}).get('href')

        elif '/book/' in url:
            pdf = bs.find('div', {'data-test': 'download-article-link-wrapper',
                                  'class': 'js-context-bar-sticky-point-desktop'}).find('a').get('href')
        if pdf is not None:
            # append base url if necessary
            if 'link.springer.com' in pdf:
                return pdf
            else:
                return f'https://link.springer.com{pdf}'
    except Exception:
        print("Error: no pdf found")
        return None

In [228]:
test_links = []
for soup, url in zip(springer_soups, springer_links):
    test_links.append(get_pdf(soup, url))

test_links


['https://link.springer.com/content/pdf/10.1007/s12525-020-00445-0.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-030-49570-1_14.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-642-22531-4.pdf',
 'https://link.springer.com/content/pdf/10.1007/978-3-030-06234-7_27.pdf']

In [229]:
journal_soup.find('div', class_='c-pdf-container').find('a', {'data-article-pdf': 'true'}).get('href')

'https://link.springer.com/content/pdf/10.1007/s12525-020-00445-0.pdf'

### Publisher

In [230]:
def get_publisher(bs):
    """
    Returns the publisher of the publication
    :param bs: Received bs of the publication
    :return: String
    """
    try:
        json_data = get_json_data(bs)
        publisher = json_data.get('publisher').get('name')
        return publisher
    except:
        print("Error: no publisher found")
        return None

In [231]:
for soup in springer_soups:
    print(get_publisher(soup))

Springer Berlin Heidelberg
Springer International Publishing
Springer Berlin Heidelberg
Springer International Publishing


### Year

In [232]:
def get_year(bs, url):
    """
    Returns the year of the publication
    :param url: URL of the publication
    :param bs: Received bs of the publication
    :return:
    """
    try:
        json_data = get_json_data(bs)
        if ('/chapter/' in url) or ('/article/' in url):
            date = json_data.get('datePublished')
            year = date.split('-')[0]  # get year from date (if date is available)
            return year
        if '/book/' in url:
            year = json_data.get('copyrightYear')
            return year

    except:
        print("Error: no year found")
        print(bs.name)
        return None

In [233]:
for soup, url in zip(springer_soups, springer_links):
    print(get_year(soup, url))


2020
2020
2011
2019


### Page ranges
For papers and contributions one can find the page range in the jsond data

#### Start page

In [234]:
def get_start_page(bs, url):
    """
    Returns the start page of the publication in the volume/journal/proceeding
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Start page of the publication
    """
    if '/book/' in url:
        return None
    if '/chapter/' in url or '/article/' in url:
        try:
            json_data = get_json_data(bs)
            start_page = json_data.get('pageStart')
            return start_page
        except:
            print("Error: no start page found")
            return None

In [235]:
for soup, url in zip(springer_soups, springer_links):
    print(get_start_page(soup, url))

523
201
None
283


#### End page

In [236]:
def get_end_page(bs, url):
    """
    Returns the end page of the publication in the volume/journal/proceeding
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: End page of the publication
    """
    if '/book/' in url:
        return None
    if '/chapter/' in url or '/article/' in url:
        try:
            json_data = get_json_data(bs)
            end_page = json_data.get('pageEnd')
            return end_page
        except:
            print("Error: no end page found")
            return None


In [237]:
for soup, url in zip(springer_soups, springer_links):
    print(get_end_page(soup, url))

545
214
None
296


### Publication type

In [238]:
def get_publication_type(bs, url):
    """
    Returns the publication type of the publication
    :param url: URL of the publication
    :param bs: Received bs of the publication
    :return: String
    """
    try:
        if '/book/' in url:
            type = bs.find('li', {'class': 'c-article-identifiers__item'}).text
        else:
            type = bs.find('li', {'class': 'c-article-identifiers__item', 'data-test': 'article-category'}).text
        return type
    except:
        print("Error: no publication type found in bs, deriving by url")
        if '/book/' in url:
            return 'Book'
        elif '/chapter/' in url:
            return 'Chapter'
        elif '/article/' in url:
            return 'Article'
        #return None

In [239]:
for soup, url in zip(springer_soups, springer_links):
    print(get_publication_type(soup, url))

Research Paper
Conference paper
Conference proceedings
Chapter


### Text of publication
Text extraction of journal paper is simple because a unique identification of the tag is possible. For other publications we have to apply further logic to extract the text only. The result is a dictionary with chapter titles as keys and the text of the chapter as value.

In [240]:
def extract_text_from_p_tags(p_tags):
    """
    Help method to extract text from multiple p-tags
    :param p_tags: List of p-tags as result of a bs search : bs4.element.ResultSet
    :return: String with total text
    """
    result_text = ''
    for p in p_tags:
        if result_text == '':
            result_text += p.text  # No break for first paragraph
        else:
            result_text += f'\n\n{p.text}'  # Break for paragraph
    return result_text


def get_full_text(bs, url):
    """
    Returns the accessible text of the publication structured in different paragraphs. This may be the total text or only a part of.
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Returns a list of ordered dictionaries. Each dictionary contains the name respectively text of the paragraph.
    """

    def __get_full_text_chapter(bs, text):
        """
        Internal method to extract text from chapter links
        :param bs:
        :param text:
        :return:
        """
        body = bs.find('div', class_='c-article-body')  # Body contains all sections
        sections = bs.findAll('section')
        len(sections)
        text_section = ''
        for section in sections:
            chapter_name = section.find('h2',
                                        class_='c-article-section__title js-section-title js-c-reading-companion-sections-item').text
            p_tags = section.find('div', class_='c-article-section__content').findAll('p')
            chapter_text = extract_text_from_p_tags(p_tags)
            text.append({
                'chapter_name': chapter_name,
                'chapter_text': chapter_text
            })
            next_sibling_id = section.find_next_sibling().attrs.get('id')

            # breaks loop when text sections are completed
            if next_sibling_id == 'MagazineFulltextChapterBodySuffix':
                break
        return text

    def __get_full_text_article(bs, text):
        """
        Internal function to extract text from an article
        :param bs:
        :param text:
        :return:
        """
        main_div = bs.find('div', class_='main-content')
        sections = main_div.findAll('section')
        for section in sections:
            chapter_name = section.h2.text
            p_tags = section.findAll('p')
            chapter_text = extract_text_from_p_tags(p_tags)
            text.append({
                'chapter_name': chapter_name,
                'chapter_text': chapter_text
            })
        return text

    text = []

    if '/article/' in url:  # article extractions works
        try:
            return __get_full_text_article(bs, text)
        except:
            print("Error try clause")

    if '/chapter/' in url:
        try:
            return __get_full_text_chapter(bs, text)
        except:
            print("Error: try clause in chapter")

    if '/book/' in url:
        return None

In [241]:
text = get_full_text(journal_soup, journal_full_link)

for chapter in text:
    chapter_name = chapter['chapter_name']
    chapter_text = chapter['chapter_text']
    print(f'Kapitel: {chapter_name} \n')
    print(chapter_text)
    print('\n \n')

Kapitel: Introduction 

How retailers and customers interact is constantly changing. The advent of omni-channel retailing made clear that customer interaction does not stop at a particular channel’s boundaries (Lemon and Verhoef 2016; Saghiri et al. 2017), as exemplified by hybrid customer interactions, where customers use digital and physical channels in parallel (Hosseini et al. 2017; Nüesch et al. 2015).

With the emergence of smart retail technologies (Roy et al. 2017) that blur the boundaries between physical and digital channels (Roy et al. 2018), the number of touchpoints (TPs) a retailer must manage is increasing (Lewis et al. 2014; von Briel 2018). While the use of information and communication technology to improve processes has a long history in brick and mortar (BaM) retail, front-stage interactions with customers have primarily been based on person-to-person interaction (Betzing et al. 2018). In contrast, e-commerce, by definition, is built upon customer contact via e-serv

In [242]:
# test
text = get_full_text(conference_chapter_soup, conference_chapter_link)

for chapter in text:
    chapter_name = chapter['chapter_name']
    chapter_text = chapter['chapter_text']
    print(f'Kapitel: {chapter_name} \n')
    print(chapter_text)
    print('\n \n')

Kapitel:  Abstract 

The identification of coordinated campaigns within Social Media is a complex task that is often hindered by missing labels and large amounts of data that have to be processed. We propose a new two-phase framework that uses unsupervised stream clustering for detecting suspicious trends over time in a first step. Afterwards, traditional offline analyses are applied to distinguish between normal trend evolution and malicious manipulation attempts. We demonstrate the applicability of our framework in the context of the final days of the Brexit in 2019/2020.

 

Kapitel: 1 Introduction 

Social media has become an important infrastructure for modern information sharing and networking. In most developed countries, the majority of people are already connected via one or multiple platforms [6]. Even more important, decision makers like politicians or multiplicators like journalists are also an integral part of social media networks. These groups function as bridge between 

In [243]:
i = 1
for bs, url in zip(springer_soups, springer_links):
    print('----------------------------------------')
    print(f'TEXT NR: {i} VON {url}')
    print('----------------------------------------')
    text = get_full_text(bs, url)
    if text is not None:
        for chapter in text:
            chapter_name = chapter['chapter_name']
            chapter_text = chapter['chapter_text']
            print(f'Kapitel: {chapter_name} \n')
            print(chapter_text)
            print('\n \n')
    i += 1

----------------------------------------
TEXT NR: 1 VON https://link.springer.com/article/10.1007/s12525-020-00445-0
----------------------------------------
Kapitel: Introduction 

How retailers and customers interact is constantly changing. The advent of omni-channel retailing made clear that customer interaction does not stop at a particular channel’s boundaries (Lemon and Verhoef 2016; Saghiri et al. 2017), as exemplified by hybrid customer interactions, where customers use digital and physical channels in parallel (Hosseini et al. 2017; Nüesch et al. 2015).

With the emergence of smart retail technologies (Roy et al. 2017) that blur the boundaries between physical and digital channels (Roy et al. 2018), the number of touchpoints (TPs) a retailer must manage is increasing (Lewis et al. 2014; von Briel 2018). While the use of information and communication technology to improve processes has a long history in brick and mortar (BaM) retail, front-stage interactions with customers have

### References
Springer often provides Google Scholar links as well

In [244]:
def get_references(bs, url):
    """
    Returns the information of the references of the publication
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Returns a list of dictionaries with the reference text and the Google Scholar link (if existing) in th form of
    {'text': 'reference text', 'link': 'link to Google Scholar'}
    """
    if '/book/' in url:
        return None

    references = []
    try:
        reference_items = bs.find_all('li', class_='c-article-references__item')
    except:
        return None  # No references found
    for item in reference_items:
        reference_text = item.find('p', class_='c-article-references__text').text
        try:
            gscholar_link = item.find('a', {'data-track-action': 'google scholar reference'}).get('href')
        except:
            gscholar_link = None
        references.append({
            'reference_text': reference_text,
            'google_scholar_link': gscholar_link
        })
    return references

In [245]:
for soup, url in zip(springer_soups, springer_links):
    try:
        print(len(get_references(soup, url)))
    except:
        print(None)

183
23
None
41


In [246]:
# len(journal_soup.find_all('li', class_='c-article-references__item js-c-reading-companion-references-item'))
# for i in get_references(conference_chapter_soup, conference_chapter_link):
#     if i['google_scholar_link'] is not None:
#         print(i['google_scholar_link'])
#         print("")

## Journal Fields
### Journal name

In [247]:
def get_journal_name(bs, url):
    """
    Returns the journal name of the publication
    Returns None if publication is not a journal article
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Journal name (String)
    """

    json_data = get_json_data(bs)
    if '/article/' in url:
        journal_name = json_data.get('isPartOf').get('name')
        return journal_name
    else:
        return None

In [248]:
for soup, url in zip(springer_soups, springer_links):
    print(get_journal_name(soup, url))

Electronic Markets
None
None
None


### Journal volume

In [249]:
def get_journal_volume(bs, url):
    """
    Returns the journal volume of the publication
    Returns None if publication is not a journal article
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Journal volume (String)
    """
    json_data = get_json_data(bs)
    if '/article/' in url:
        journal_volume = json_data.get('isPartOf').get('volumeNumber')
        return journal_volume
    else:
        return None

In [250]:
for soup, url in zip(springer_soups, springer_links):
    print(get_journal_volume(soup, url))

32
None
None
None


## Conference fields
### Conference name

In [251]:
def get_conference_name(bs, url):
    if get_publication_type(bs, url) == 'Conference paper':
        conference_name = bs.find('p', class_='c-chapter-info-details u-mb-8').find('a', {
            'data-track': 'click', 'data-track-action': 'open conference'
        }).text
        return conference_name
    else:
        return None

In [252]:
for soup, url in zip(springer_soups, springer_links):
    print(get_conference_name(soup, url))

None
International Conference on Human-Computer Interaction
None
None


### Venue -> omitted

### Conference proceeding/ Book title

In [253]:
def get_proceedings(bs, url):
    """
    Returns the title of the conference proceedings or book under which the publication was published.
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Title of the proceedings/book (String)
    """
    if get_publication_type(bs, url) == 'Conference paper':
        try:
            json_data = get_json_data(bs)
            proceedings = json_data.get('isPartOf').get('name')
            return proceedings
        except:
            return None
    else:
        return None

In [254]:
for soup, url in zip(springer_soups, springer_links):
    print(get_proceedings(soup, url))

None
Social Computing and Social Media. Design, Ethics, User Behavior, and Social Network Analysis
None
None


## Book/ volume contributions
### Book title

In [255]:
def get_book_title(bs, url):
    """
    Returns the title of the book under which the publication was published.
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Title of the book (String)
    """
    if get_publication_type(bs, url) == 'Chapter':
        try:
            json_data = get_json_data(bs)
            book_title = json_data.get('isPartOf').get('name')
            return book_title
        except:
            return None
    else:
        return None

In [256]:
for soup, url in zip(springer_soups, springer_links):
    print(get_book_title(soup, url))

None
None
None
The Art of Structuring


In [257]:
def get_editors(bs, url):
    """
    Returns the editors of the volume (under which the publication was published)
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: List of Editors : [String]
    """
    if '/chapter/' in url:
        try:
            editor_div = bs.find('div', {'class': 'c-article-section__content', 'id': 'editor-information-content'})
            editors = []
            for editor in editor_div.find_all('p', class_='c-article-author-affiliation__authors-list'):
                editors.append(editor.text)
            # remove titles since we scrape the names
            # remove Everything including the point from strings in list
            editors = [editor.split('.')[1].strip() for editor in editors]
            return editors
        except:
            return None
    # in books the editors are in the json file
    if '/book/' in url:
        try:
            json_data = get_json_data(bs)
            editors = [editor.get('name') for editor in json_data.get('editor')]
            return editors
        except:
            return None
    else:
        return None


In [258]:
for soup, url in zip(springer_soups, springer_links):
    print(get_editors(soup, url))

None
['Gabriele Meiselwitz']
['Herbert Kuchen']
['Katrin Bergener', 'Michael Räckers', 'Armin Stein']


## Books (in general: proceedings or editor volumes)
### Book subtitle

In [259]:
def get_book_subtitle(bs, url):
    """
    Returns the subtitle of the book (proceedings or editor volume).
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Subtitle of the book (String)
    """
    if '/book/' in url:
        try:
            json_data = get_json_data(bs)
            book_subtitle = json_data.get('alternateName')
            return book_subtitle
        except:
            return None
    return None

In [260]:
for soup, url in zip(springer_soups, springer_links):
    print(get_book_subtitle(soup, url))

None
None
20th International Workshop, WFLP 2011, Odense, Denmark, July 19, 2011, Proceedings
None


## Springer Metrics
### Accesses

In [261]:
# Navigating with parent because accesses and citations do not have individual features.

def get_accesses(bs, url):
    """
    Returns the number of accesses of the publication according to the Springer metric.
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Number of accesses (String) #TODO change to int
    """
    try:
        accesses = bs.find('span', text='Accesses').parent.text.strip().split(' ')[0]  # navigating up in the tree
        return accesses
    except:
        print("error")
        return None


In [262]:
for soup, url in zip(springer_soups, springer_links):
    print(get_accesses(soup, url))

3844
3894
3080
2349


### Citations

In [263]:
# Navigating with parent because accesses and citations do not have individual features.

def get_citations(bs, url):
    """
    Returns the number of citations of the publication according to the Springer metric.
    :param bs: Received bs of the publication
    :param url: URL of the publication
    :return: Number of citations (String) #TODO change to int
    """
    try:
        citations = bs.find('span', text='Citations').parent.text.strip().split(' ')[0]  # navigating up in the tree
        return citations
    except:
        try:
            # Citation
            citations = bs.find('a', class_='c-article-metrics-bar__label',
                                text='Citations').parent.text.split(' ')[0].strip()
            return citations
        except:
            return None

In [264]:
for soup, url in zip(springer_soups, springer_links):
    print(get_citations(soup, url))

5
2
46
13
